In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import os
import requests
import re
import time
from time import sleep
            

# 크롬 드라이버 다운로드 및 자동 설정
chrome_driver_path = ChromeDriverManager().install()

# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 불필요한 에러 메시지 삭제
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# 크롬 브라우저를 열고 cbs news 웹페이지로 이동
region = '광진구음식점'
url = f"https://map.naver.com/p/search/{region}"
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=chrome_options)
driver.get(url)
# driver.maximize_window()
time.sleep(10)

In [2]:
# iframe으로 전환
iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
driver.switch_to.frame(iframe)
time.sleep(3)
print("iframe 전환")

iframe 전환


In [3]:
def scroll_down():
    # 한 번에 10개의 식당이 나오고, 스크롤 시 동적으로 추가되어 총 50개까지 한 페이지 안에 나온다
    for i in range(4):
        # 페이지 아래로 스크롤
        driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container"))
        time.sleep(3)
    


In [4]:
def next_page():

    # '>' 버튼 찾기
    next_button = driver.find_elements(By.CSS_SELECTOR, ".eUTV2")[-1]
    if next_button.get_attribute("aria-disabled") == 'true':
        print("Last Page")
        return -1

    else:
        next_button.click()
        print("click")

    time.sleep(3)

# for i in range(4):
#     tag = next_page()
#     if tag == -1:
#         break


In [5]:
# driver.switch_to.default_content()

from selenium.webdriver.common.by import By

# 현재 프레임을 변경한 적이 있는지 여부를 확인
try:
    driver.find_element(By.XPATH, ".//*")
    print("현재 프레임: 최상위 컨텍스트")
except:
    # 만약 예외가 발생한다면 현재 프레임이 설정된 것임
    print("현재 프레임:", driver.current_frame)


현재 프레임: 최상위 컨텍스트


In [85]:
driver.switch_to.default_content()
# iframe 나가기

In [86]:
iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
rst_taps = driver.switch_to.frame(iframe)

# 레스토랑 탭 들어가기

In [28]:
# 식당정보 수집
def crawl_rst_info():
    try:
        rst_name = driver.find_element(By.CSS_SELECTOR, ".Fc1rA").text              # 식당이름
    except:
        rst_name = None
    try:
        rst_category =  driver.find_element(By.CSS_SELECTOR, ".DJJvD").text         # 식당 카테고리
    except:
        rst_category = None
    try:
        rst_nreviews =  driver.find_elements(By.CSS_SELECTOR, ".PXMot")[-2].text    # 식당 리뷰 수
    except:
        rst_nreviews = None
    try:
        rst_address =  driver.find_element(By.CSS_SELECTOR, ".LDgIH").text          # 식당 주소
    except:
        rst_address = None
    try:
        rst_number = driver.find_element(By.CSS_SELECTOR, ".xlx7Q").text            # 식당 전화번호
    except:
        rst_number = None

    time_button = driver.find_elements(By.CSS_SELECTOR, "._UCia")[1].click()        # 식당 시간 더보기 버튼 클릭
    time.sleep(3)
    rst_times =  driver.find_elements(By.CSS_SELECTOR, ".A_cdD")[1:]                # 요일별 영업시간들
    times = []
    for rst_time in rst_times:
        times.append(rst_time.text)



    tabs = driver.find_elements(By.CSS_SELECTOR, "._tab-menu")
    for tab in tabs:
        if tab.text == '메뉴':
            menu_button = tab                                                       # 메뉴 정보 수집

    menu_button.click()
    time.sleep(1)
    menus_tab = driver.find_elements(By.CSS_SELECTOR, ".E2jtL")
    menus = {}
    for menu in menus_tab:
        title = menu.find_element(By.CSS_SELECTOR, ".lPzHi").text
        price = menu.find_element(By.CSS_SELECTOR, ".GXS1X")
        price = price.find_element(By.TAG_NAME, "em").text                          # 가격 정보 함께 수집

        menus[title] = price


    print(rst_name)       # clear
    # print(rst_category)   # clear
    # print(rst_nreviews)   # clear
    # print(rst_address)    # clear
    # print(rst_number)     # clear
    # print(times)          # clear
    # print(menus)          # clear

    rst_info = [rst_name, rst_category, rst_address, times, rst_nreviews, rst_number, menus, 0]

    return rst_info

In [87]:
def find_show_more():
    try:
        # '더보기' 버튼을 찾음
        show_more_button = WebDriverWait(driver, 2).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.TeItc'))
        )
        # 버튼 클릭
        show_more_button.click()
        time.sleep(2)
    except:
        # 더 이상 '더보기' 버튼이 없으면 반복 종료
        return

def crawl_review(driver, rst_name):
    user = driver.find_element(By.CSS_SELECTOR, ".P9EZi").text                                  # 유저 이름

    time.sleep(1)

    # 리뷰 전체보기 클릭
    try:
        more_button = driver.find_element(By.CSS_SELECTOR, ".WPk67")
        more_button.click()
    except:
        pass
    
    # 리뷰가 적혀있지 않은 경우, None으로 처리
    try:
        review_text = driver.find_element(By.CSS_SELECTOR, ".zPfVt").text.replace("\n", " ")    # 리뷰 텍스트
    except:
        review_text = None
    
    u_rst_tag = []
    try:
        tags = driver.find_elements(By.CSS_SELECTOR, ".sIv5s")                                  # 유저가 리뷰에 남긴 태그
        for tag in tags:
            u_rst_tag.append(tag.text)
    except:
        pass
    
    try:
        date = driver.find_element(By.CSS_SELECTOR, ".CKUdu time").text                         # 리뷰 남긴 날짜
    except:
        date = None

    result = [rst_name, user, review_text, u_rst_tag, None, date, 0]                            # 리스트로 저장

    return result

def crawl_review_info(rst_name):
    tabs = driver.find_elements(By.CSS_SELECTOR, "._tab-menu")
    for tab in tabs:
        if tab.text == '리뷰':
            review_button = tab
            break                                                                                   # 리뷰 정보 수집
    review_button.click()
    time.sleep(3)
    
    for i in range(1):
        find_show_more()
    # while True:
    #     try:
    #         find_show_more()                                                          # 끝까지 '더보기' 버튼 클릭
    #         print("리뷰 로딩중")
    #     except:
    #         break
    
    print("리뷰 전체 로드 완료")

    results = []                                                                                    # 레스토랑 하나 당 결과를 담을 리스트
    reviews = driver.find_elements(By.CSS_SELECTOR, ".owAeM")
    for review in reviews[:4]:
        result = crawl_review(review, rst_name)
        results.append(result)

        print(result)
        time.sleep(1)
    
    return results  

    

In [89]:
rst_test = driver.find_elements(By.CSS_SELECTOR, ".UEzoS")[5]
rst = rst_test.find_element(By.CSS_SELECTOR, ".tzwk0")

# 레스토랑 하나 테스트 용으로 방문
rst.click()
time.sleep(1)

# 전체 페이지로 복귀하도록 iframe에서 벗어나기 -> 그래야만 새로 생긴 iframe 탭에 접근할 수 있음
driver.switch_to.default_content()

# 새로 생긴 식당 탭 iframe으로 전환
iframe = driver.find_element(By.CSS_SELECTOR, "#entryIframe")
driver.switch_to.frame(iframe)
time.sleep(1)
# print("식당 탭 iframe 전환")

"""
    
정보 수집
"""
results = []

info = crawl_rst_info()
time.sleep(1)
results.extend(crawl_review_info(info[0]))



IndexError: list index out of range

In [91]:
print(results[1:])

[['이가네양꼬치 구의역점', '춘삼이일', '양갈비 추천해주셨는데 진짜 맛있고 직원분들 친절해서 너무 좋아요!!!', ['음식이 맛있어요', '고기 질이 좋아요', '친절해요'], None, '23.12.24.일', 0], ['이가네양꼬치 구의역점', '다이렉트맨19', '', ['음식이 맛있어요', '고기 질이 좋아요', '양이 많아요', '특별한 메뉴가 있어요', '가성비가 좋아요'], None, '3.14.목', 0], ['이가네양꼬치 구의역점', 'Cefg', '사장님 정말 친절하시고 고기도 잘 구워주셨어요 !! 반찬리필도 말 안해도 가져다주시고 마지막에 마파두부도 주시는데.. 이가네 중에 구의역점 서비스 최고에요..', ['음식이 맛있어요', '고기 질이 좋아요', '양이 많아요', '친절해요', '직접 잘 구워줘요'], None, '23.10.22.일', 0]]


In [ ]:
rsts = driver.find_elements(By.CSS_SELECTOR, ".UEzoS")

for rst in rsts:
    rst = rst.find_element(By.CSS_SELECTOR, ".tzwk0")

    # 레스토랑 방문
    rst.click()
    time.sleep(2)

    # 전체 페이지로 복귀하도록 iframe에서 벗어나기 -> 그래야만 새로 생긴 iframe 탭에 접근할 수 있음
    driver.switch_to.default_content()

    # 새로 생긴 식당 탭 iframe으로 전환
    iframe = driver.find_element(By.CSS_SELECTOR, "#entryIframe")
    driver.switch_to.frame(iframe)
    time.sleep(2)
    print("식당 탭 iframe 전환")

    """
    
    정보 수집
    """
    crawl_rst_info()

    # iframe 다시 레스토랑 전체 탭으로 복귀
    driver.switch_to.default_content()
    iframe = driver.find_element(By.CSS_SELECTOR, "iframe#searchIframe")
    driver.switch_to.frame(iframe)
    time.sleep(2)
    

In [78]:
rst_time = driver.find_elements(By.CSS_SELECTOR, ".A_cdD")
for time in rst_time:
    print(time.text)

In [77]:
rst_time

[]

In [50]:
print(rst_name)
print(rst_category)
print(rst_nreviews)   # clear
print(rst_address)    # clear
print(times)

불야성
요리주점
서울 광진구 동일로22길 89 1층
방문자리뷰 87
['영업 중\n08:00에 영업 종료\n8시 0분에 영업 종료', '매일\n17:00 - 08:00']


In [22]:
for time in rst_time:
    print(time.text)

영업 중
08:00에 영업 종료
8시 0분에 영업 종료
매일
17:00 - 08:00


In [1]:
name = 'abc'
number = 123
a = [name, number]

In [2]:
a

['abc', 123]

In [10]:
test = []
a = [['a', 'b', 'c'], ['a', 'b', 'c']]
b = [['d', 'e', 'f']]

test.extend(a)
test.extend(b)
print(test)

[['a', 'b', 'c'], ['a', 'b', 'c'], ['d', 'e', 'f']]


In [92]:
def remove_emoji(text):
    # 이모지 패턴 정규 표현식
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # 스마일 이모티콘
                               u"\U0001F300-\U0001F5FF"  # 이모티콘 및 기호
                               u"\U0001F680-\U0001F6FF"  # 기호
                               u"\U0001F1E0-\U0001F1FF"  # 국기 표시 (1)
                               "]+", flags=re.UNICODE)
    # 이모지 제거
    return emoji_pattern.sub(r'', text)

In [93]:
remove_emoji("Hello! 😊 This is a test string with emojis! 🚀")

'Hello!  This is a test string with emojis! '

In [2]:
import pandas as pd
test = pd.DataFrame(data=[['a','b']])

In [3]:
test

,0,1
0,a,b


In [5]:
import os

# 저장할 파일의 경로 설정
folder_path = "E:/MOON/capstone_data"  # E 드라이브의 MOON 폴더 경로

file_path = os.path.join(folder_path + "/test2.csv")

# 데이터프레임을 CSV 파일로 저장
test.to_csv(file_path, encoding='utf-8-sig', index=False)


In [6]:
print("dd\naa")

dd
aa
